# LSSTCam pointing errors
# Looking at the encoder offsets.

Craig Lage - 02-Jul-25

In [ ]:
import os
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
client = makeEfdClient()
eventMaker = TMAEventMaker()

# Now look at how the offset varies with Az/El

In [ ]:
dayObs = 20250629
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Offset_vs_AzEl{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 29 seconds")
# This weeds out slews with next to zero motion.
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    #print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")

azs = []
els = []
azOffsets = []
elOffsets = []
seqNums = []

for event in goodEvents:
    seqNums.append(event.seqNum)
    axs = []
    axs.append(fig.add_subplot(1,2,1))
    axs.append(fig.add_subplot(1,2,2))
    plt.subplots_adjust(wspace = 0.5)
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )

    elPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_elevationInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_azimuthInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    # This stops the plot when the axis is in position    
    if len(azPos) > 0:
        azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
    azValues = np.asarray(az["actualPosition"])
    azs.append(azValues[0])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    if len(elPos) > 0:
        elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
    elValues = np.asarray(el["actualPosition"])
    els.append(elValues[0])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[0].scatter(azValues, azEncInterp)
    azFit = np.polyfit(azValues, azEncInterp, 1)
    #print(fit)
    axs[0].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0].transAxes)
    axs[0].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0].transAxes)
    azOffsets.append((azFit[1]*3600))
    axs[0].set_xlabel("Azimuth (degrees)")
    axs[0].set_ylabel("Azimuth Encoder average(degrees)")
    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].scatter(elValues, elEncInterp)
    elFit = np.polyfit(elValues, elEncInterp, 1)
    #print(fit)
    axs[1].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1].transAxes)
    axs[1].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1].transAxes)
    elOffsets.append((elFit[1]*3600))
    axs[1].set_xlabel("Elevation (degrees)")
    axs[1].set_ylabel("Elevation Encoder average(degrees)")
    pdf.savefig(fig)
    fig.clf()
pdf.close()  

fig = plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Encoder offsets {dayObs}")
plt.subplot(1,2,1)
plt.scatter(azs, azOffsets)
plt.ylim(-500,500)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Azimuth offset (arcseconds)")
plt.subplot(1,2,2)
plt.scatter(els, elOffsets)
plt.ylim(-800,800)
plt.xlabel("Elevation(degrees)")
plt.ylabel("Elevation offset (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Offset_Summary_{dayObs}.png")